In [1]:
import os

In [2]:
os.chdir('../')


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path



In [33]:
from cnn_classification import *
from cnn_classification.utils.common import read_yaml, create_directory
from cnn_classification.constants import *


In [42]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_NAME, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig: 
        config = self.config.data_ingestion 
        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

    

In [36]:
import os 
import urllib.request as request
import zipfile
from cnn_classification import logger
from cnn_classification.utils.common import get_size

In [48]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file,) 
            logger.info(f"Downloaded {filename} ")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        if not os.path.exists(self.config.unzip_dir):
            os.makedirs(self.config.unzip_dir)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(self.config.unzip_dir)
            logger.info(f"Extracted files to {self.config.unzip_dir}")
        else:
            logger.info(f"Directory already exists: {self.config.unzip_dir}")        
        

In [49]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    logger.info(f"Data Ingestion Config: {data_ingestion_config}")
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Error during data ingestion: {e}")
    raise e

2025-03-31 18:36:41,538 - INFO - common- YAML file loaded successfully: config/config.yaml
2025-03-31 18:36:41,542 - INFO - common- YAML file loaded successfully: params.yaml
2025-03-31 18:36:41,543 - INFO - common- Directory created: artifacts
2025-03-31 18:36:41,544 - INFO - common- Directory created: artifacts/data_ingestion
2025-03-31 18:36:41,544 - INFO - 2892529653- Data Ingestion Config: DataIngestionConfig(root_dir='artifacts/data_ingestion', source_url='https://github.com/ad2546/Data/raw/refs/heads/main/archive%20(8).zip', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion/unzipped')
2025-03-31 18:36:41,545 - INFO - 172851876- File already exists


2025-03-31 18:36:41,653 - INFO - 172851876- Extracted files to artifacts/data_ingestion/unzipped
